# Anotação automática do tipo de resposta

Este notebook documenta o processo de anotação automática do tipo de resposta,
realizado como primeira etapa do projeto. O objetivo é explicitar como
rótulos automáticos foram gerados e posteriormente combinados com revisão
manual para a obtenção da anotação final utilizada nos experimentos de PLN.

Esta etapa não corresponde ainda à modelagem supervisionada final, mas sim
a um procedimento de apoio à construção do corpus anotado.


## Contexto metodológico

O projeto investiga tipos de resposta quanto ao grau de informatividade
(*subinformativa*, *completa*, *sobreinformativa*). Antes da modelagem
supervisionada, foi implementada uma anotação automática preliminar, baseada
em heurísticas linguísticas simples, com o objetivo de acelerar o processo
de anotação e identificar casos prototípicos e casos limítrofes.


In [1]:
import pandas as pd


## Carregamento dos dados 

Utilizei dois conjuntos de dados:

- um corpus completo anotado automaticamente quanto ao tipo de resposta;
- uma amostra desse corpus submetida a revisão manual (≈30% dos dados).

A integração entre esses conjuntos é feita por meio de um identificador único (`id`).


In [93]:
# corpus completo
auto = pd.read_csv("corpus_anotado_v1.csv")

# amostra revisada manualmente
rev = pd.read_csv("amostra_revisao_manual.csv")

print(auto.shape)
print(rev.shape)



(7, 6)
(90, 8)


## Estrutura do corpus bruto

Verifica-se a estrutura do corpus bruto, assegurando a presença das colunas
necessárias para a anotação automática.


In [70]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             300 non-null    int64 
 1   pergunta       300 non-null    object
 2   resposta       300 non-null    object
 3   classificação  300 non-null    object
dtypes: int64(1), object(3)
memory usage: 9.5+ KB


## Heurística de anotação automática

A anotação automática foi realizada com base em uma heurística simples,
inspirada em pressupostos teóricos da informatividade: respostas mais longas
tendem a ser sobreinformativas, respostas muito curtas tendem a ser
subinformativas, e respostas intermediárias tendem a ser completas.

Essa heurística não pretende capturar inferências pragmáticas complexas,
servindo apenas como ponto de partida para a anotação manual posterior.


In [72]:
import re
import pandas as pd

def limpar(texto):
    return re.sub(r"[^\w\sáéíóúãõç]", "", texto.lower()).strip()
def resposta_minima_completa(pergunta, resposta):
    resposta = resposta.strip().lower()
    pergunta = pergunta.lower()

    if resposta not in ["sim", "não"]:
        return False

    if " e " in pergunta or " ou " in pergunta:
        return False

    return True
def extrair_verbo_principal(pergunta):
    pergunta = limpar(pergunta)
    palavras = pergunta.split()

    # heurística simples: primeiro verbo flexionado comum
    for p in palavras:
        if p.endswith(("a","e","am","em","ou","eu","ia","ava","aram","eram","iram")):
            return p
    return None
def extrair_entidades(pergunta):
    texto = limpar(pergunta)

    # corta após o verbo para evitar ruído
    texto = re.split(
        r"\bfoi|foram|fala|falam|chegou|chegaram|participou|participaram|entregou|entregaram\b",
        texto
    )[0]

    partes = [p.strip() for p in texto.split(" e ")]

    stopwords = {
        "o","a","os","as","um","uma","do","da","de","no","na",
        "para","ao","à","dos","das"
    }

    entidades = []
    for p in partes:
        tokens = [t for t in p.split() if t not in stopwords]
        if tokens:
            entidades.append(tokens[-1])  # núcleo do SN

    return entidades
def classificar_informatividade(pergunta, resposta):
    pergunta_norm = limpar(pergunta)
    resposta_norm = limpar(resposta)

    #REGRA 0: resposta mínima ("sim" / "não") ---
    if resposta_minima_completa(pergunta_norm, resposta_norm):
        return "completa"

    #REGRA 1: resposta elíptica afirmativa ("sim, fala") ---
    verbo = extrair_verbo_principal(pergunta)
    if verbo:
        if resposta_norm.startswith(("sim","claro","com certeza","óbvio","aham")) and verbo in resposta_norm:
            return "completa"

    #REGRA 2: extrai entidades ---
    entidades = extrair_entidades(pergunta)
    num_total = len(entidades)

    mencionadas = [
        e for e in entidades if re.search(rf"\b{e}\b", resposta_norm)
    ]
    num_mencionadas = len(mencionadas)

    conjuncao_e = " e " in pergunta_norm

    quantificadores = ["todos","todas","ninguém","nenhum","todo mundo"]
    tem_quantificador = any(q in resposta_norm for q in quantificadores)

    # --- SUBINFORMATIVA ---
    if conjuncao_e and 0 < num_mencionadas < num_total:
        return "subinformativa"

    # --- SOBREINFORMATIVA ---
    if tem_quantificador:
        return "sobreinformativa"

    # --- COMPLETA explícita ---
    if conjuncao_e and num_mencionadas == num_total:
        return "completa"

    # --- COMPLETA simples (1 entidade) ---
    if num_total == 1 and num_mencionadas == 1:
        return "completa"

    return "indefinida"

dados = [
    ["O João e a Maria foram à festa?", "A Maria foi."],
    ["O João foi à festa?", "Todos foram."],
    ["O João e a Maria foram à festa?", "O João e a Maria foram."],
    ["A Maria fala japonês?", "Sim."],
    ["A Maria fala japonês?", "Sim, fala."],
    ["O João e a Maria foram à festa?", "Sim."],
    ["Os alunos e os professores participaram?", "Os professores participaram."],
]

df = pd.DataFrame(dados, columns=["pergunta","resposta"])
df["categoria"] = df.apply(
    lambda x: classificar_informatividade(x["pergunta"], x["resposta"]),
    axis=1
)

print(df)



                                   pergunta                      resposta  \
0           O João e a Maria foram à festa?                  A Maria foi.   
1                       O João foi à festa?                  Todos foram.   
2           O João e a Maria foram à festa?       O João e a Maria foram.   
3                     A Maria fala japonês?                          Sim.   
4                     A Maria fala japonês?                    Sim, fala.   
5           O João e a Maria foram à festa?                          Sim.   
6  Os alunos e os professores participaram?  Os professores participaram.   

          categoria  
0    subinformativa  
1  sobreinformativa  
2          completa  
3          completa  
4          completa  
5        indefinida  
6    subinformativa  


## Aplicação da anotação automática

A heurística definida é aplicada a todas as respostas do corpus bruto,
resultando em uma coluna de rótulos automáticos.


In [73]:
df["classe_auto"] = df.apply(
    lambda row: classificar_informatividade(row["pergunta"], row["resposta"]),
    axis=1
)

df.head()


,pergunta,resposta,categoria,classe_auto
0,O João e a Maria foram à festa?,A Maria foi.,subinformativa,subinformativa
1,O João foi à festa?,Todos foram.,sobreinformativa,sobreinformativa
2,O João e a Maria foram à festa?,O João e a Maria foram.,completa,completa
3,A Maria fala japonês?,Sim.,completa,completa
4,A Maria fala japonês?,"Sim, fala.",completa,completa


## Distribuição das classes automáticas

A distribuição dos rótulos automáticos é analisada para verificar tendências
gerais e identificar possíveis vieses introduzidos pela heurística.


In [74]:
df["classe_auto"].value_counts()


classe_auto
completa            3
subinformativa      2
sobreinformativa    1
indefinida          1
Name: count, dtype: int64

## Limitações da anotação automática

A anotação automática baseada apenas no comprimento da resposta ignora fatores
semânticos e pragmáticos relevantes, como implicaturas, pressuposições e
adequação ao contexto da pergunta. Por esse motivo, seus resultados não são
utilizados diretamente na modelagem final, sendo sempre submetidos a revisão
manual.


## Revisão manual e anotação final

Após a anotação automática, uma amostra do corpus foi revisada manualmente,
resultando em ajustes dos rótulos e na identificação de casos ambíguos,
posteriormente classificados como *indefinidos*.

A anotação final (`classe_final`) resulta da combinação entre anotação
automática, revisão manual e critérios teóricos.


In [75]:
df_manual = pd.read_csv("amostra_revisao_manual_revisada.csv")
df_manual.head()


,id,pergunta,resposta,classificação,classe_auto,anotacao_manual,correta,observacoes
0,204,O Caio gostou da viagem?,"Sim, gostou.",Completa,completa,completa,sim,NaN
1,267,O noivo pagou o casamento?,Pagou.,Completa,indefinida,completa,não,NaN
2,153,O Mário comprou a moto?,Comprou.,Completa,indefinida,completa,não,NaN
3,10,O Rafael e a Laura participaram da reunião?,O Rafael participou.,Subinformativa,subinformativa,subinformativa,sim,NaN
4,234,A Gabi sabe andar a cavalo?,Sabe.,Completa,indefinida,completa,não,NaN


## Integração das anotações

Nesta etapa, as anotações automáticas e manuais são integradas, resultando
na anotação final utilizada nos experimentos subsequentes.


In [76]:
def normalizar_texto(s):
    return (
        s.astype(str)
         .str.strip()
         .str.replace(r"\s+", " ", regex=True)
    )

for col in ["pergunta", "resposta"]:
    df[col] = normalizar_texto(df[col])
    df_manual[col] = normalizar_texto(df_manual[col])


In [84]:
df = df.merge(
    df_manual[["pergunta", "resposta", "anotacao_manual"]],
    on=["pergunta", "resposta"],
    how="left"
)


In [83]:
df["anotacao_manual"].notna().value_counts(normalize=True)


anotacao_manual
False    1.0
Name: proportion, dtype: float64

In [79]:
df["classe_final"] = df["anotacao_manual"].fillna(df["classe_auto"])


## Classe *indefinida*

Durante a revisão manual, alguns casos foram classificados como *indefinidos*,
por apresentarem ambiguidades ou por não se enquadrarem claramente nos critérios
definidos para as demais classes. Esses casos são mantidos no corpus final
para fins descritivos, mas excluídos da modelagem automática inicial.


In [80]:
df["classe_final"].value_counts()

classe_final
completa            3
subinformativa      2
sobreinformativa    1
indefinida          1
Name: count, dtype: int64

In [81]:
df.to_csv("corpus_anotado_v1.csv", index=False)


## Conclusão

Este notebook documentou o processo de anotação automática e revisão manual
empregado na construção do corpus anotado. A anotação final resultante serve
como base para a etapa seguinte do projeto, dedicada a realizar a análise exploratória do corpus anotado.
